In [1]:
from src.hpc_env import HPCenv
from src.baseline import MedianBaseline 
from src.utils import get_config_as_dict, VideoGenerator, mask_fn

In [4]:
import gymnasium as gym
import numpy as np
import configparser
import ast
import os

from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

In [5]:
# Load config with explicit path and typed parsing
config = configparser.ConfigParser()
config_path = os.path.join(os.getcwd(), 'config_file', 'config.ini')
config.read(config_path)
config_dict = get_config_as_dict(config)

#### Masking function size

In [6]:
WORKLOAD_PATH = "data/workloads/lublin_256.swf"
env = HPCenv(workload_path=WORKLOAD_PATH, config=config)
masking_function = mask_fn(env)

assert (env.action_space_size == len(masking_function))
print("Masking function test passed ✅")

Max Allocated Processors: 256 ;max node: 256 ;max procs: 256 ;max execution time: 124707
Masking function test passed ✅


#### Observation state 

In [5]:
env = HPCenv(workload_path=WORKLOAD_PATH, config=config)
env.reset(seed=42,options=None)

predefined_size = env.observation_space._shape
actual_shape = env.build_observation().shape

print("Predefined sized:", predefined_size)
print("Actual observation state size: ", actual_shape)
assert predefined_size == actual_shape
print("Observation state size passed ✅")

Max Allocated Processors: 256 ;max node: 256 ;max procs: 256 ;max execution time: 124707
Predefined sized: (1439,)
Actual observation state size:  (1439,)
Observation state size passed ✅


#### Carbon intensity forecast size and prediction

In [6]:
carbon_encoding = env.carbon_intensity.create_carbon_forecast_enconding(current_timestamp=0)

assert config_dict['green_feature_constant'] + (config_dict['green_forecast_length']-1)*config_dict['green_feature_pr_timeslot'] == len(carbon_encoding)


print("Carbon encoding size test passed ✅")

Carbon encoding size test passed ✅


#### Baseline test

In [8]:
seed = 2
render_env = HPCenv(workload_path=WORKLOAD_PATH,generate_rendering=True, name="median_baseline", config=config)
baseline = MedianBaseline(env=render_env, config=config)
reward = baseline.run(seed=seed)
print(reward)

Max Allocated Processors: 256 ;max node: 256 ;max procs: 256 ;max execution time: 124707
Directory created successfully at 'renderings/median_baseline/seed_2'.
-1823157.492043055


## Generate rendering videos


In [ ]:
video_generator = VideoGenerator(path=render_env.dir_path)
video_generator.generate_video()

MoviePy - Building video rendering.mp4.
MoviePy - Writing video rendering.mp4



MoviePy - Done !
MoviePy - video ready rendering.mp4
Video saved as  median_baseline/seed_2/*.pngrendering.my


#### Config reader

In [ ]:
get_config_as_dict(config=config)

{'use_constant_power': True,
 'constant_power_per_processor': 500,
 'procs_per_node': 1,
 'idle_power': 15.0,
 'carbon_year': 2021,
 'variable_carbon_intensities': True,
 'green_forecast_length': 24,
 'eta': 0.0,
 'max_queue_size': 256,
 'run_win_length': 64,
 'delay_time_list': [300, 600, 1200, 1800, 2400, 3000, 3600],
 'delay_time_list_length': 7,
 'max_wait_n_jobs': 10,
 'job_feature': 5,
 'run_feature': 2,
 'green_feature_pr_timeslot': 1,
 'green_feature_constant': 8,
 'episode_length': 256,
 'gamma': 1.0,
 'gae_lambda': 0.97,
 'base_line_wait_carbon_penality': 0.01,
 'bounded_slowdown_threshhold': 10,
 'reward_type': 'CO2_direct',
 'max_power': 19000,
 'max_green': 19000,
 'max_wait_time': 200000,
 'max_run_time': 162754,
 'max_requested_processors': 256}